# Основной код

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
full_data = pd.read_csv('data.csv')
full_data

,ФИО,текстовый редактор,телефон,вино,ос,напиток
0,Воробьева Арина Сергеевна,jupyter,android,никакое,windows,кофе
1,Макарова Анна Евгеньевна,pycharm,iphone,никакое,windows,кофе
2,Мирзагаламов Ринат Радикович,pycharm,iphone,никакое,mac,чай
3,Олейников Иван Андреевич,vim,android,белое,linux,чай
4,Романадзе Екатерина Левановна,sublime text,iphone,никакое,linux,чай
5,Мохов Алексей Игоревич,vs code,iphone,никакое,linux,чай
6,Чернева Надежда,vs code,android,белое,mac,кофе
7,Иванов Андрей Алексеевич,vs code,iphone,никакое,mac,кофе
8,Еремеев Даниил Дмитриевич,sublime text,iphone,белое,mac,чай
9,Лебедев Евгений,pycharm,android,красное,windows,кофе


In [3]:
class Decision_tree:
    def __init__(self, full_data, target):
        tree = dict()
        train_data = full_data.drop(columns=['ФИО'])
        
        def addLayer(obj, data):
            ent = self.entropy_of_all(data.drop(columns=[target]))
            field, ent_value = max(ent, key=lambda x: x[1])
            if ent_value == 0:
                targets = data[target].value_counts()
                max_count = max(dict(targets).items(), key=lambda x: x[1])[1]
                answer = [val for val, c in dict(targets).items() if c == max_count]
                obj[target] = ' или '.join(answer)
                return

            obj[field] = dict()
            for v in data[field].unique():
                obj[field][v] = dict()
                target_values = data.loc[data[field] == v][target].value_counts()
                if len(target_values) == 1 or ent_value == 0:
                    obj[field][v][target] = max(dict(target_values), key=lambda x: x[1])
                else:
                    addLayer(obj[field][v], data.loc[data[field] == v].drop(columns=[field]))

        addLayer(tree, train_data)

        self.tree = tree
        self.target = target
    
    def entropy(self, df, attribute):
        s = df.nunique()[attribute]
        n = len(df)
        m = df[attribute].value_counts().values
        return - sum(m/n * np.log(m/n)) 

    def entropy_of_all(self, df):
        gains = []
        for attribute in df.columns:
            gains.append((attribute, self.entropy(df, attribute)))
        return gains
        
    def predict(self, sample):
        def nextLayer(obj, sample):
            field = list(obj.keys())[0]
            if field == self.target:
                return obj[field]
            value = sample[field]
            return nextLayer(obj[field][value], sample)

        return nextLayer(self.tree, sample)
    
    def show_tree(self):
        print(json.dumps(self.tree, indent=2, ensure_ascii=False))

# Использование

## Дерево для определения текстового редактора

In [4]:
text_editor_tree = Decision_tree(full_data, 'текстовый редактор')

In [5]:
sample_text_editor = {
    'напиток': 'чай',
    'телефон': 'iphone',
     'вино': 'никакое',
     'ос': 'mac'
}

In [6]:
text_editor_tree.predict(sample_text_editor)

'pycharm'

In [7]:
text_editor_tree = Decision_tree(full_data, 'текстовый редактор')

In [8]:
sample_text_editor = {
    'напиток': 'чай',
    'телефон': 'iphone',
     'вино': 'никакое',
     'ос': 'mac'
}

In [9]:
text_editor_tree.predict(sample_text_editor)

'pycharm'

## Дерево для определения вина

In [10]:
vine_tree = Decision_tree(full_data, 'вино')

In [11]:
sample_vine = {
    'напиток': 'чай',
    'телефон': 'iphone',
     'текстовый редактор': 'vim',
     'ос': 'linux'
}

In [12]:
vine_tree.predict(sample_vine)

'никакое или белое'

In [13]:
vine_tree.show_tree()

{
  "текстовый редактор": {
    "jupyter": {
      "вино": "никакое"
    },
    "pycharm": {
      "ос": {
        "windows": {
          "телефон": {
            "iphone": {
              "вино": "никакое"
            },
            "android": {
              "вино": "красное"
            }
          }
        },
        "mac": {
          "напиток": {
            "чай": {
              "вино": "никакое"
            },
            "кофе": {
              "вино": "красное"
            }
          }
        }
      }
    },
    "vim": {
      "вино": "никакое или белое"
    },
    "sublime text": {
      "ос": {
        "linux": {
          "напиток": {
            "чай": {
              "вино": "никакое"
            },
            "кофе": {
              "вино": "красное"
            }
          }
        },
        "mac": {
          "вино": "белое"
        }
      }
    },
    "vs code": {
      "телефон": {
        "iphone": {
          "вино": "никакое"
        },
        "android"